# Vertex AI MLOps Book - Chapter 6 - Big Query ML - Credit Card Default Prediction

<table align="left">
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw..ipynb" target="_blank">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI icon">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>

## Dataset - Google Cloud BQ Public Dataset - Credit Card Default

**Description** - Obfuscated Google Analytics 360 data. It’s a great way to look at business data and experiment and learn the benefits of analyzing Google Analytics 360 data in BigQuery. Learn more at: https://support.google.com/analytics/answer/7586738

**Location** - bigquery-public-data.ml_datasets.credit_card_default

**Objective** - Predict the probability of a website visitor will complete a purchase transaction

#### Install the required libraries

In [ ]:
! pip3 install google-cloud-aiplatform

In [ ]:
! pip3 install google-cloud-bigquery

In [ ]:
! pip3 install db-dtypes

#### Uncomment the following cell if running notebook locally. Not required if running on Vertex AI Workbench

In [31]:
#!gcloud auth login

Import the required libraries

In [2]:
from google.cloud import bigquery
client = bigquery.Client()

/Users/jasmeetbhatia/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


## Set GCP Project ID

Set your GCP Project ID. Replace 'jsb-alto' with your project ID.

In [20]:
Project_ID = 'jsb-alto'

# Set the project id
! gcloud config set project {Project_ID}

Updated property [core/project].


#### Let's first create the BigQuery dataset where the model will reside

In [ ]:
# Define the query statement
query = f"""
CREATE SCHEMA `{Project_ID}.credit_default_dataset`
"""

# Create the dataset
dataset_status = client.query(query).to_dataframe()


""


## Look at the Credit Card data we will be using in this notebook

Let's first look at the fields of the dataset. Description field below provides details about each field.

In [16]:
query = f"""
SELECT 
table_name,column_name,	data_type,	description	
FROM `bigquery-public-data.ml_datasets.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`
WHERE table_name = 'credit_card_default'
"""

data_schema = client.query(query).to_dataframe()
data_schema

,table_name,column_name,data_type,description
0,credit_card_default,id,FLOAT64,Anonymized ID of each client
1,credit_card_default,limit_balance,FLOAT64,Amount of given credit in NT dollars (includes...
2,credit_card_default,sex,STRING,"Gender (1=male, 2=female)"
3,credit_card_default,education_level,STRING,"Education Level (1=graduate school, 2=universi..."
4,credit_card_default,marital_status,STRING,"Marital status (1=married, 2=single, 3=others)"
5,credit_card_default,age,FLOAT64,Age in years
6,credit_card_default,pay_0,FLOAT64,"Repayment status in September, 2005 (-1=pay du..."
7,credit_card_default,pay_2,FLOAT64,"Repayment status in August, 2005 (scale same a..."
8,credit_card_default,pay_3,FLOAT64,"Repayment status in July, 2005 (scale same as ..."
9,credit_card_default,pay_4,FLOAT64,"Repayment status in June, 2005 (scale same as ..."


## Let's look at a sample of the data

In [8]:
# Define the query
query = f"""
SELECT
  *
FROM
  `bigquery-public-data.ml_datasets.credit_card_default` 
  LIMIT 10
"""

# Run the query, and return a pandas DataFrame
data = client.query(query).to_dataframe()

# Preview the data
data

,id,limit_balance,sex,education_level,marital_status,age,pay_0,pay_2,pay_3,pay_4,...,bill_amt_6,pay_amt_1,pay_amt_2,pay_amt_3,pay_amt_4,pay_amt_5,pay_amt_6,default_payment_next_month,predicted_default_payment_next_month,id_str
0,27502.0,80000.0,1,6,1,54.0,0.0,0.0,0.0,0.0,...,17643.0,2545.0,2208.0,1336.0,2232.0,542.0,348.0,1,"[{'tables': {'score': 0.8667634129524231, 'val...",27502
1,26879.0,200000.0,1,4,1,49.0,0.0,0.0,0.0,0.0,...,48984.0,1689.0,2164.0,2500.0,3480.0,2500.0,3000.0,0,"[{'tables': {'score': 0.9351968765258789, 'val...",26879
2,18340.0,20000.0,2,6,2,22.0,0.0,0.0,0.0,0.0,...,0.0,4641.0,1019.0,900.0,0.0,1500.0,0.0,1,"[{'tables': {'score': 0.8572560548782349, 'val...",18340
3,13692.0,260000.0,2,4,2,33.0,0.0,0.0,0.0,0.0,...,29890.0,5000.0,5000.0,1137.0,5000.0,1085.0,5000.0,0,"[{'tables': {'score': 0.9690881371498108, 'val...",13692
4,20405.0,150000.0,1,4,2,32.0,0.0,0.0,0.0,-1.0,...,146411.0,4019.0,146896.0,157436.0,4600.0,4709.0,5600.0,0,"[{'tables': {'score': 0.9349926710128784, 'val...",20405
5,3882.0,300000.0,2,4,2,32.0,0.0,0.0,0.0,0.0,...,700.0,15235.0,1491.0,1303.0,0.0,2000.0,1400.0,0,"[{'tables': {'score': 0.9530552625656128, 'val...",3882
6,7227.0,130000.0,1,1,1,45.0,0.0,0.0,0.0,0.0,...,65099.0,2886.0,2908.0,2129.0,2354.0,2366.0,2291.0,0,"[{'tables': {'score': 0.9030028581619263, 'val...",7227
7,1379.0,200000.0,1,1,1,58.0,0.0,0.0,0.0,0.0,...,129167.0,7822.0,4417.0,4446.0,4597.0,4677.0,4698.0,0,"[{'tables': {'score': 0.8636506199836731, 'val...",1379
8,29477.0,500000.0,1,1,1,39.0,0.0,0.0,0.0,0.0,...,204975.0,54209.0,4607.0,4603.0,5224.0,207440.0,7509.0,0,"[{'tables': {'score': 0.9399265050888062, 'val...",29477
9,10643.0,230000.0,1,1,1,48.0,0.0,0.0,0.0,0.0,...,110094.0,7000.0,6607.0,3773.0,4290.0,4164.0,2000.0,0,"[{'tables': {'score': 0.8917242884635925, 'val...",10643


## Split data into Train, Validate and Test sets 

Use Farm_Fingerprint to split the data into 80% train, 10% validate and 10% test sets. New column called 'data_split' is added to the dataset to indicate the split. We Store the data in a new view called 'credit_data_view'

In [24]:
query = f"""
CREATE OR REPLACE VIEW {Project_ID}.credit_default_dataset.credit_data_view
AS SELECT *
FROM 
(
WITH
  base_table AS (
  SELECT
    *,
  FROM
    `bigquery-public-data.ml_datasets.credit_card_default`)

SELECT
  *, 
  CASE
    WHEN ABS(MOD(FARM_FINGERPRINT(CAST(id AS STRING)), 10)) < 8 THEN 'train'
    WHEN ABS(MOD(FARM_FINGERPRINT(CAST(id AS STRING)), 10)) = 8 THEN 'validation'
    WHEN ABS(MOD(FARM_FINGERPRINT(CAST(id AS STRING)), 10)) = 9 THEN 'test'
END
  AS data_split
FROM
  bigquery-public-data.ml_datasets.credit_card_default) o

"""

data_split = client.query(query).to_dataframe()
data_split

""


### Create Training Dataset
Let's create the query to select the fields we want to use as features in our Classification model. 
We will not include the 'id' field as it is just an identifier and not useful for our model or the predicted fields included in the raw data. We will also filter on the 'data_split' field to only include the training data.

In [27]:

train_data_query = f"""
SELECT
  id, limit_balance,sex,education_level,marital_status,age,pay_0,pay_2,pay_3,pay_4,bill_amt_5,bill_amt_6,
  pay_amt_1,pay_amt_2,pay_amt_3,pay_amt_4,pay_amt_5,pay_amt_6,default_payment_next_month
FROM
  `{Project_ID}.credit_default_dataset.credit_data_view` 
WHERE data_split = 'train'
"""
# Run the query
data = client.query(train_data_query).to_dataframe()

# Preview the data
data

,id,limit_balance,sex,education_level,marital_status,age,pay_0,pay_2,pay_3,pay_4,bill_amt_5,bill_amt_6,pay_amt_1,pay_amt_2,pay_amt_3,pay_amt_4,pay_amt_5,pay_amt_6,default_payment_next_month
0,27502.0,80000.0,1,6,1,54.0,0.0,0.0,0.0,0.0,26210.0,17643.0,2545.0,2208.0,1336.0,2232.0,542.0,348.0,1
1,26879.0,200000.0,1,4,1,49.0,0.0,0.0,0.0,0.0,50235.0,48984.0,1689.0,2164.0,2500.0,3480.0,2500.0,3000.0,0
2,18340.0,20000.0,2,6,2,22.0,0.0,0.0,0.0,0.0,500.0,0.0,4641.0,1019.0,900.0,0.0,1500.0,0.0,1
3,20405.0,150000.0,1,4,2,32.0,0.0,0.0,0.0,-1.0,143375.0,146411.0,4019.0,146896.0,157436.0,4600.0,4709.0,5600.0,0
4,7227.0,130000.0,1,1,1,45.0,0.0,0.0,0.0,0.0,63832.0,65099.0,2886.0,2908.0,2129.0,2354.0,2366.0,2291.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2339,3982.0,20000.0,1,3,2,26.0,-1.0,-1.0,-1.0,-2.0,0.0,0.0,1560.0,0.0,0.0,0.0,0.0,0.0,0
2340,13325.0,80000.0,2,3,2,28.0,-1.0,-1.0,-1.0,-2.0,0.0,0.0,2800.0,0.0,0.0,0.0,0.0,0.0,0
2341,139.0,50000.0,2,3,1,51.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,300.0,5880.0,0.0,0.0,0.0,0.0,1
2342,26185.0,450000.0,2,2,1,38.0,-2.0,-2.0,-2.0,-2.0,390.0,390.0,390.0,780.0,390.0,390.0,390.0,390.0,1


## Train a Classification Model

### Create the BQML Model

It's time to train our classification model! We'll use logistic regression for this task.

Model Type = logistic regression

Model name = credit_default_classification_model

In [28]:
query = f"""
CREATE OR REPLACE MODEL `{Project_ID}.credit_default_dataset.credit_default_classification_model`

OPTIONS(model_type='logistic_reg') AS

SELECT
  limit_balance,sex,education_level,marital_status,age,pay_0,pay_2,pay_3,pay_4,bill_amt_5,bill_amt_6,
  pay_amt_1,pay_amt_2,pay_amt_3,pay_amt_4,pay_amt_5,pay_amt_6,
  default_payment_next_month AS label
FROM
  `{Project_ID}.credit_default_dataset.credit_data_view` 
WHERE data_split = 'train'

"""

model_status = client.query(query).to_dataframe()




## Evaluate Model

Now let's evaluate our model's performance using validation data.


In [30]:
query = f"""
SELECT
  *
FROM
  ml.EVALUATE(MODEL `{Project_ID}.credit_default_dataset.credit_default_classification_model`, (
SELECT
  limit_balance,sex,education_level,marital_status,age,pay_0,pay_2,pay_3,pay_4,bill_amt_5,bill_amt_6,pay_amt_1,pay_amt_2,pay_amt_3,pay_amt_4,pay_amt_5,pay_amt_6,
  default_payment_next_month AS label
FROM
  `{Project_ID}.credit_default_dataset.credit_data_view` 
WHERE data_split = 'validation'
  
  ))
"""

model_eval = client.query(query).to_dataframe()
model_eval



,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.689655,0.238095,0.781437,0.353982,0.487995,0.745148


## Generate predictions based on the model

Finally, let's use our trained model to make predictions on new, unseen data.

In [39]:
query = f"""
SELECT
  *
FROM
  ml.PREDICT(MODEL `{Project_ID}.credit_default_dataset.credit_default_classification_model`, (
SELECT
  id,limit_balance,sex,education_level,marital_status,age,pay_0,pay_2,pay_3,pay_4,bill_amt_5,bill_amt_6,
  pay_amt_1,pay_amt_2,pay_amt_3,pay_amt_4,pay_amt_5,pay_amt_6
FROM
  `{Project_ID}.credit_default_dataset.credit_data_view` 
WHERE data_split = 'test'
  ))

"""

predictions = client.query(query).to_dataframe()
predictions

,predicted_label,predicted_label_probs,id,limit_balance,sex,education_level,marital_status,age,pay_0,pay_2,pay_3,pay_4,bill_amt_5,bill_amt_6,pay_amt_1,pay_amt_2,pay_amt_3,pay_amt_4,pay_amt_5,pay_amt_6
0,0,"[{'label': '1', 'prob': 0.06009251589061463}, ...",13692.0,260000.0,2,4,2,33.0,0.0,0.0,0.0,0.0,30767.0,29890.0,5000.0,5000.0,1137.0,5000.0,1085.0,5000.0
1,0,"[{'label': '1', 'prob': 0.062284388070289964},...",3882.0,300000.0,2,4,2,32.0,0.0,0.0,0.0,0.0,-450.0,700.0,15235.0,1491.0,1303.0,0.0,2000.0,1400.0
2,0,"[{'label': '1', 'prob': 0.10921178839652788}, ...",16919.0,320000.0,1,1,1,51.0,0.0,0.0,0.0,0.0,253903.0,249866.0,20373.0,13000.0,11160.0,8418.0,8325.0,8602.0
3,0,"[{'label': '1', 'prob': 0.07031414915307155}, ...",6144.0,470000.0,1,1,2,35.0,0.0,0.0,0.0,0.0,254175.0,256613.0,15107.0,13111.0,10000.0,10014.0,9598.0,10044.0
4,0,"[{'label': '1', 'prob': 0.07857767432050183}, ...",27314.0,310000.0,1,1,2,45.0,0.0,0.0,0.0,0.0,293350.0,300623.0,10010.0,11500.0,11015.0,11000.0,12000.0,23035.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,0,"[{'label': '1', 'prob': 0.06330068729890127}, ...",19339.0,200000.0,2,5,2,48.0,0.0,0.0,-2.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
283,0,"[{'label': '1', 'prob': 0.16944606832869455}, ...",13071.0,500000.0,2,1,1,38.0,1.0,-2.0,-1.0,-1.0,0.0,2199.0,0.0,2331.0,0.0,0.0,2199.0,0.0
284,0,"[{'label': '1', 'prob': 0.19327966498571797}, ...",2434.0,100000.0,2,1,2,36.0,1.0,-1.0,-1.0,-2.0,0.0,0.0,780.0,0.0,0.0,7600.0,0.0,0.0
285,0,"[{'label': '1', 'prob': 0.06998717132492231}, ...",21252.0,360000.0,1,1,2,39.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,1448.0,3740.0,0.0,0.0,0.0,3942.0


In the resulting table, the predicted_label column contains the predicted values for 'default_payment_next_month'. The predicted_label_probs column contains the 
probability that the predicted_label is correct. 

Congratulations on reaching the end of this chapter! We've successfully explored the dataset, prepared the data, trained a logistic regression model, evaluated it, and generated predictions—all using BigQuery ML.